## 12.1 Description

A signature algorithm is the public-key equivalent of a message authentication code. It consists of three parts:
1. a key generation algorithm, which can be **shared** with other public-key algorithms
2. a signature **generation** algorithm
3. a signature **verification** algorithm

## 12.3 DSA

The Digital Signature Algorithm (DSA) is a US Federal Government standard for digital signatures. It was first proposed by the National Institute of Standards and Technology (NIST) in 1991, to be used in the Digital Signature Standard (DSS).

> note: FIPS 186-5 (2023) the DSA is deprecated

DSA key generation happens in two steps. The first step is a choice of parameters, which can be shared between users. The second step is the generation of public and private keys for a single user.

### Parameter generation

We start by picking an approved cryptographic hash function $H$. We also pick a key length $L$ and a prime length $N$. Next we choose a prime $q$ of length $N$ bits; $N$ must be less than or equal to the length of the hash output. We also pick an $L$-bit prime $p$ such that $p − 1$ is a multiple of $q$.

The last part is the most confusing. We have to find a number $g$ whose *multiplicative order* $\pmod p$ is $q$. The easy way to do this is to set $$g \equiv 2^{(p−1)/q} \pmod p$$

Once we have parameters $(p, q, g)$, they can be shared between users.

### Key generation

First, select a random $x$ with $0 < x < q$. Next, calculate $y$ where $y \equiv g^x \pmod p$. This delivers a public key $(p, q, g, y)$, and private key $x$.

### Signing a message

In order to sign a message, the signer picks a random $k$ between $0$ and $q$.
With $k$ chosen, they then compute the two parts of the signature $r, s$ of the message $m$:
$$r \equiv (g^k \pmod p) \pmod q$$
$$s \equiv k^{-1}(H(m) + xr) \pmod q$$


### Verifying a signature

Verifying the signature is a lot more complex. Given the message $m$ and signature $(r, s)$:
$$w \equiv s^{-1} \pmod q$$
$$u_1 \equiv H(m)w \pmod q$$
$$u_2 \equiv rw \pmod q$$
$$v \equiv (g^{u_1}y^{u_2} \pmod p) \pmod q$$
If the signature is valid that final result $v$ will be equal to $r$, the second part of the signature.

### The trouble with $k$

In particular, the choice of the signature parameter $k$ is critical. DSA's requirements for the $k$ value are a combination of all of these:
- It has to be unique.
- It has to be unpredictable.
- It has to be secret.

Suppose that an attacker sees multiple signatures $(r_i, s_i)$, for different messages $m_i$, all with the same $k$. The attacker picks any two signatures $(r_1, s_1)$ and $(r_2, s_2)$ of messages $m_1$
and $m_2$ respectively. Writing down the equations for $s_1$ and
$s_2$:

$$s_1 \equiv k^{-1}(H(m_1) + xr_1) \pmod q$$
$$s_2 \equiv k^{-1}(H(m_2) + xr_2) \pmod q$$


The attacker can simplify this further: $ r_1 $ and $ r_2 $ must be
equal, following the definition: 

$$
r_i \equiv g^k \pmod{q}
$$

Since the signer is reusing $ k $, and the value of $ r $ only depends on $ k $, all $ r_i $ will be equal. Since the signer is using the same key, $ x $ is equal in the two equations as well.